### Wczytanie danych z biblioteki TorchVision

In [ ]:
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data import DataLoader


# transformacje dla pre-processingu
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.286,), (0.353,))
    ]
)

# FashionMNIST dataset
train_dataset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Data loader
batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### EDA

In [ ]:
from matplotlib import pyplot as plt
LABELS = {
  0: "T-shirt/top",
  1: "Trouser",
  2: "Pullover",
  3: "Dress",
  4: "Coat",
  5: "Sandal",
  6: "Shirt",
  7: "Sneaker",
  8: "Bag",
  9: "Ankle boot"
}

fig, axes = plt.subplots(ncols=3, nrows=5, figsize=(4, 5))
axes = axes.flatten()
for i,(img, label) in enumerate(train_dataset):

  if i >= 15: break

  axes[i].imshow(img[0, :, :], cmap="gray")
  axes[i].axis("off")
  axes[i].set_title(LABELS[label])

plt.tight_layout()
plt.show()

### Definiowane dwóch modeli konwolucyjnych

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18, ResNet18_Weights

class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
        self.pool = nn.AvgPool2d(2, 2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.fc1 = nn.Linear(16*5*5, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model_custom = CustomCNN()

model_resnet = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
# Zmiana pierwszej warstwy konwolucyjnej na 1 kanał ze dla zdjęć GRAYSCALE
model_resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)

### Definiowanie parametrów treningu

In [ ]:
from torch import optim
from tqdm import tqdm

def train(model, save_name, epochs=5):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    for epoch in range(epochs):
        losses = []
        for i, (inputs, labels) in tqdm(enumerate(train_dataloader, 0)):

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

        # print loss
        print(f'{epoch + 1}/{epochs+1} | loss: {np.round(np.mean(losses), 4)}')

    torch.save(model.state_dict(), save_name)

In [ ]:
train(model_custom, 'model_custom.pth')

In [ ]:
train(model_resnet, 'model_resnet.pth')

In [ ]:
del model_custom, model_resnet

### Walidacja modelu na zbiorze testowym

In [ ]:
# 1.6. Czytanie modelu

model_custom = CustomCNN()
model_resnet = resnet18()
# Zmiana pierwszej warstwy konwolucyjnej na 1 kanał ze dla zdjęć GRAYSCALE
model_resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)

model_custom.load_state_dict(torch.load("model_custom.pth", weights_only=True))
model_resnet.load_state_dict(torch.load("model_resnet.pth", weights_only=True))

In [ ]:
def evaluate(model):
    counter_labels = 0
    counter_correct_preds = 0

    model.eval()
    with torch.no_grad():
        for images, labels in test_dataloader:

            predicted = torch.max(model(images).data, 1)[1]

            counter_labels += labels.size(0)
            counter_correct_preds += (predicted == labels).sum().item()

    accuracy = np.round(counter_correct_preds * 100 / counter_labels, 3)

    print(f'Global accuracy on test set: {accuracy}%')

In [ ]:
evaluate(model_custom)

In [ ]:
evaluate(model_resnet)

In [ ]:
def confusion_matrix(model, n_classes=10):
    matrix = np.zeros((n_classes, n_classes),dtype=int)

    model.eval()
    with torch.no_grad():
        for images, labels in test_dataloader:

            predicted = torch.max(model(images).data, 1)[1]

            for p,l in zip(predicted, labels):
                p,l = p.item(), l.item()
                matrix[p,l] += 1
    
    return matrix

In [ ]:
matrix_custom = confusion_matrix(model_custom)

In [ ]:
matrix_resnet = confusion_matrix(model_resnet)

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(matrix_custom, cmap='ocean')
ax[0].set_title("Custom CNN")
ax[0].set_xticks(list(LABELS.keys()))
ax[0].set_yticks(list(LABELS.keys()))

ax[0].set_ylabel("Predicted")
ax[0].set_xlabel("Label")

for i in range(10):
    for j in range(10):
        ax[0].text(j, i, matrix_custom[i, j], ha="center", va="center", color="black", fontsize=8)


ax[1].imshow(matrix_resnet, cmap='ocean')
ax[1].set_title("ResNet18")
ax[1].set_xticks(list(LABELS.keys()))
ax[1].set_yticks(list(LABELS.keys()))

ax[1].set_ylabel("Predicted")
ax[1].set_xlabel("Label")

for i in range(10):
    for j in range(10):
        ax[1].text(j, i, matrix_resnet[i, j], ha="center", va="center", color="black", fontsize=8)

plt.show()

In [ ]:
def analyze_matrix(matrix):
    global_res = {"accuracy": np.zeros(matrix.shape[0]), "precision": np.zeros(matrix.shape[0]), "recall": np.zeros(matrix.shape[0]), "f1": np.zeros(matrix.shape[0])}
    for i,c in LABELS.items():
        print(f'{c}:')
        tp = matrix[i,i]
        fp = np.sum(matrix[i,:]) - tp
        fn = np.sum(matrix[:,i]) - tp
        tn = np.sum(matrix) - tp - fp - fn

        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * precision * recall / (precision + recall)
        accuracy = (tp+tn) / (tp+fp+fn+tn)

        global_res["accuracy"][i] = accuracy
        global_res["precision"][i] = precision
        global_res["recall"][i] = recall
        global_res["f1"][i] = f1

        print(f'\taccuracy: {accuracy*100.0:.2f}, precision: {precision*100.0:.2f}, recall: {recall*100.0:.2f}, F1: {f1*100.0:.2f}')
    
    print(f'-------------\nGlobal (macro) results:')
    print(f'\taccuracy: {np.mean(global_res["accuracy"])*100.0:.2f}, precision: {np.mean(global_res["precision"])*100.0:.2f}, recall: {np.mean(global_res["recall"])*100.0:.2f}, F1: {np.mean(global_res["f1"])*100.0:.2f}')
    



In [ ]:
analyze_matrix(matrix_custom)

In [ ]:
analyze_matrix(matrix_resnet)